# Download data

In [ ]:
import os
import zipfile
import kaggle
# use kaggle api to download the data or manully download the data urself
##! Kaggle competitions download -c "jane-street-real-time-market-data-forecasting" 

# make the directory to be "data/jane-street-real-time-market-data-forecasting" and extract file there u f the code dont work do it manually
#3os.makedirs("data/jane-street-real-time-market-data-forecasting", exist_ok=True) # create directory for jane-street-real-time-market-data-forecasting-housing
##!unzip jane-street-real-time-market-data-forecasting.zip -d data/jane-street-real-time-market-data-forecasting/

jane-street-real-time-market-data-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)


'unzip' is not recognized as an internal or external command,
operable program or batch file.


# Kaggle Competition 

In [ ]:
import numpy as np
import pandas as pd
import polars as pl

In [86]:
# function for import the data .parquet to pandas dataframe
def download_data(i): # the data have 10 partiton 0 to 9
    file_path = f"data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
    df = pd.read_parquet(file_path)
    print(f"import train data part {i}")
    return df

In [87]:
# test/example of the import fuction
train0 = download_data(0)

In [88]:
# import test data 
file_path  = f"data/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet"
test = pd.read_parquet(file_path)

# data preprocessing

In [117]:
#check train data 1
train0.tail(20)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
1944190,169,847,38,1.570022,NaN,NaN,NaN,NaN,NaN,-0.013255,...,-0.139359,0.578174,0.094325,-1.442982,-0.352376,-0.190914,0.029033,0.255568,0.183223,0.556433
1944191,169,848,0,1.740154,NaN,NaN,NaN,NaN,NaN,-0.019650,...,-0.177105,-1.959539,-1.218961,-2.282917,-1.477447,-0.836411,-3.480990,-0.208067,-0.054775,-0.461447
1944192,169,848,1,2.671190,NaN,NaN,NaN,NaN,NaN,-0.014743,...,-0.263282,0.708025,0.122007,-0.168188,0.306298,0.244731,0.447845,0.029364,0.048978,0.049966
1944193,169,848,2,1.517848,NaN,NaN,NaN,NaN,NaN,-0.020840,...,0.183500,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,0.444804,0.207303,0.902221
1944194,169,848,3,0.953731,NaN,NaN,NaN,NaN,NaN,-0.021532,...,1.224004,0.748528,0.518571,1.776199,-0.236258,-0.078679,0.805653,-0.343752,-0.130761,-0.732494
1944195,169,848,7,1.789037,NaN,NaN,NaN,NaN,NaN,-0.014222,...,-0.252050,2.690954,1.854106,5.000000,1.295612,0.370915,2.651581,-1.344183,-0.526427,-2.111102
1944196,169,848,8,1.665622,NaN,NaN,NaN,NaN,NaN,-0.027113,...,-0.213265,-1.952719,0.753256,1.753474,0.753926,0.347689,1.002481,-0.953408,-0.330281,-0.981245
1944197,169,848,9,1.892650,NaN,NaN,NaN,NaN,NaN,-0.024533,...,-0.154888,1.453674,0.949433,-1.063989,0.738098,0.540964,0.030697,-0.017320,0.019186,-0.019747
1944198,169,848,10,1.244815,NaN,NaN,NaN,NaN,NaN,-0.024115,...,-0.259167,1.014766,1.044241,5.000000,0.914982,0.644817,5.000000,0.138480,0.103964,0.237636
1944199,169,848,12,1.853232,NaN,NaN,NaN,NaN,NaN,-0.023897,...,-0.153412,5.000000,3.718975,5.000000,5.000000,2.820848,5.000000,-0.891958,-0.361855,-1.227063


In [120]:
df = pd.DataFrame()
df['gg'] = list(zip(train0['date_id'], train0['time_id'], train0['symbol_id']))
print(df['gg'].value_counts())

gg
(0, 0, 1)         1
(124, 141, 3)     1
(124, 141, 34)    1
(124, 141, 33)    1
(124, 141, 19)    1
                 ..
(70, 674, 13)     1
(70, 674, 12)     1
(70, 674, 10)     1
(70, 674, 9)      1
(169, 848, 38)    1
Name: count, Length: 1944210, dtype: int64


In [116]:
print(np.NaN in train0.feature_06.unique())

False


In [99]:
train0.symbol_id.unique()

array([ 1,  7,  9, 10, 14, 16, 19, 33,  0,  2, 13, 15, 38,  3, 12,  8, 17,
       34, 11, 30], dtype=int8)

In [37]:
#check test data
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 85 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   row_id      39 non-null     int64  
 1   date_id     39 non-null     int16  
 2   time_id     39 non-null     int16  
 3   symbol_id   39 non-null     int8   
 4   weight      39 non-null     float32
 5   is_scored   39 non-null     bool   
 6   feature_00  39 non-null     float32
 7   feature_01  39 non-null     float32
 8   feature_02  39 non-null     float32
 9   feature_03  39 non-null     float32
 10  feature_04  39 non-null     float32
 11  feature_05  39 non-null     float32
 12  feature_06  39 non-null     float32
 13  feature_07  39 non-null     float32
 14  feature_08  39 non-null     float32
 15  feature_09  39 non-null     float64
 16  feature_10  39 non-null     float64
 17  feature_11  39 non-null     float64
 18  feature_12  39 non-null     float32
 19  feature_13  39 non-null     flo

# model 1 

# Export data

In [79]:
# export data code from competition provider

lags_ : pl.DataFrame | None = None

# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 1 minute of the batch features being provided.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_
    if lags is not None:
        lags_ = lags

    # Replace this section with your own predictions
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    # Confirm has as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

In [85]:
# the run time is too long but might just needed to be reset

inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            'jane-street-real-time-market-data-forecasting/test.parquet',
            'jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

c:\Users\Asus\OneDrive\เอกสาร\2025\JaneStreet_Kaggle\kaggle_evaluation\core\templates.py:128: RuntimeWarning: 30739 seconds elapsed before server startup.
                This exceeds the startup time limit of 900 seconds that the gateway will enforce
                during the rerun on the hidden test set. Start the server before performing any time consuming steps.
  warnings.warn(
